<a href="https://colab.research.google.com/github/FANG87003/2025_DL_hws/blob/main/hw5-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 113-2 深度學習 作業 5-2

## 1 . 將存好的模型和訓練權重讀回來用

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/My Drive/Colab Notebooks'

/content/drive/My Drive/Colab Notebooks


In [3]:
from tensorflow.keras.models import model_from_json

with open('SaveModel/Imdb_LSTM_model.json') as f:
  model = model_from_json(f.read())
model.load_weights('SaveModel/Imdb_LSTM_model.weights.h5')

### model.summary()

In [4]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,235,141 (16.16 MB)

 Trainable params: 1,411,713 (5.39 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,823,428 (10.77 MB)

## 2 . 載入自行建立的tokenizer

In [5]:
import pickle

with open('SaveModel/imdb_tokenizer_10000.pkl', 'rb') as f:
  tokens = pickle.load(f)

In [6]:
from tensorflow.keras.preprocessing import sequence

## 3 . 輸入實際由IMDb取得的影評

In [7]:
input_text = '''I had a fun time watching the show and it's nice to see some riders from previous show in this season. I think the acting is good, the story is good, writing is good and I like most characters. My main problem like Ryuki is the ending, I hate the fact that I need to watch a special to conclude the story but the story overall isn't bad to be honest. I enjoyed Decade for the most part and I recommend it.'''

## 4 . 轉換影評文字串為數字串，並截長或補短，使數字串長度為100

In [8]:
input_seq = tokens.texts_to_sequences([input_text])

In [9]:
len(input_seq[0])

81

In [10]:
pad_input_seq = sequence.pad_sequences(input_seq, maxlen=100)

In [11]:
len(pad_input_seq[0])

100

In [12]:
pad_input_seq

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    9,   65,    3,
         249,   54,  145,    1,  119,    2,   41,  323,    5,   63,   45,
        8231,   35,  956,  119,    7,   10,  807,    9,  100,    1,  112,
           6,   48,    1,   61,    6,   48,  483,    6,   48,    2,    9,
          36,   87,  101,   57,  289,  435,   36,    6,    1,  273,    9,
         777,    1,  188,   11,    9,  355,    5,  102,    3,  314,    5,
        5697,    1,   61,   17,    1,   61,  443,  214,   75,    5,   25,
        1198,    9,  506, 2064,   14,    1,   87,  169,    2,    9,  382,
           8]], dtype=int32)

In [13]:
# 測試將影評數字串還原為文字串
pad_input_text = tokens.sequences_to_texts(pad_input_seq)

In [14]:
pad_input_text

["i had a fun time watching the show and it's nice to see some riders from previous show in this season i think the acting is good the story is good writing is good and i like most characters my main problem like is the ending i hate the fact that i need to watch a special to conclude the story but the story overall isn't bad to be honest i enjoyed decade for the most part and i recommend it"]

## 5 . 輸入影評(數字串，長度為100)進行情意預測

In [15]:
predict_result = model.predict(pad_input_seq)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [16]:
predict_result

array([[0.99992025]], dtype=float32)

In [17]:
predict_result[0][0]

np.float32(0.99992025)

In [18]:
SentimentDict = {1:'正面的', 0:'負面的'}

In [19]:
SentimentDict[round(predict_result[0][0])]

'正面的'

## 6 . 建立predict_review函數：

In [35]:
from tensorflow.keras.preprocessing import sequence

def predict_review(input_text):
  # 將文字轉成數字序列
  input_seq = tokens.texts_to_sequences([input_text])
  pad_input_seq = sequence.pad_sequences(input_seq, maxlen=100)

  # 預測機率
  predict_result = model.predict(pad_input_seq)
  pred = predict_result[0][0]

  # 判斷情緒
  SentimentDict = {1:'正面的', 0:'負面的'}
  sentiment = SentimentDict[round(pred)]

  print(f"預測結果：{sentiment}（{pred}）")
  return sentiment, pred

## 7 . 影評情意預測

### (1) IMDb 影評 1

In [36]:
# URL: https://www.imdb.com/title/tt1473235/review/rw5985910/?ref_=tturv_1

text1 = """I had a fun time watching the show and it's nice to see some riders from previous show in this season. I think the acting is good, the story is good, writing is good and I like most characters. My main problem like Ryuki is the ending, I hate the fact that I need to watch a special to conclude the story but the story overall isn't bad to be honest. I enjoyed Decade for the most part and I recommend it."""

predict_review(text1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
預測結果：正面的（0.9999202489852905）


('正面的', np.float32(0.99992025))

### (2) IMDb 影評 2

In [37]:
# URL: https://www.imdb.com/title/tt1473235/review/rw7592756/?ref_=tturv_6

text2 = """I'm confused by people who answered that this series is a series with a good story, good plot, good characters, etc.
For good characters, I agree, Tsukasa Kadoya's character is a character that has good character development, and this is the only good thing about this series.
I think Kamen Rider Ghost is much better than this."""

predict_review(text2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
預測結果：負面的（0.19336891174316406）


('負面的', np.float32(0.19336891))

### (3) IMDb 影評 3

In [38]:
# URL: https://www.imdb.com/title/tt1473235/review/rw2125994/?ref_=tturv_4

text3 = """When I first know about the story, I'm really excited to watch this one. Imagine, a world where all kamen raider that ever existed, fight with each other and eventually lead to the destruction of themselves by an individual called Kamen Raider Decade.
Well, I must admit that I really like the story. Although I'm a little bit confused by Decade's design (which I thought to be the most ridiculous and the weirdest I've ever seen), it's still OK.
So, I guess this is another movie that's quite entertaining to see. A lot of Kamen Raider (actually, all of the new generation Kamen Raider and a few addition) will make an appearance here and you will be quite surprise about what you'll find here, because I do.
A twisted plot, mixed world, mixed heroes and mixed villains; all of them have become something that's worth watching. Try and watch it."""

predict_review(text3)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
預測結果：正面的（0.9999878406524658）


('正面的', np.float32(0.99998784))

## 8 . 用 Gradio 打造情意分析 Web App

In [24]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.4 MB/s eta 0:00:00


In [39]:
import gradio as gr

def imdb_score(text):
  sentiment, pred = predict_review(text)
  return f"{sentiment}（{pred}）"

iface = gr.Interface(imdb_score, inputs="textbox", outputs="text", title="IMDb 情意分析")
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c15ce51a900d171144.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
